In [1]:
from guide_calling import *
import os

%matplotlib inline  
%load_ext autoreload
%autoreload 2

sns.set_style('white')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
sample='904_3_v3'
guide_calls='guide_calls/'
output_dir='output/'

In [3]:
##load guide barcode reads from outs/guide_barcode_reads.txt.gz file in experiment folder
gbc_reads = pd.read_table('data/904_3_v3.guide_barcode_reads.txt', header=None, names=('guide_identity', 'cell_barcode', 'UMI'))
cell_barcodes = pd.read_csv('data/barcodes.tsv.gz', sep='\t', header=None, names=['cell_barcode'])

In [4]:
##remove unmapped reads
##this is a conservative step
##in some cases, these unmapped reads represent real UMIs/CBCs but fail to map
gbc_reads=gbc_reads[gbc_reads['guide_identity']!='*']

##collect reads that come from valid cell barcodes as determined by scRNAseq
captured_gbc_table = capture_reads(gbc_reads, cell_barcodes)

Filtering by cell barcode...


In [5]:
##determine thresholds
os.system('[[ -d '+guide_calls+' ]] || mkdir '+guide_calls)
pop=pd.DataFrame()
for guide in captured_gbc_table.reset_index()['guide_identity'].unique():
    pop=pop.append(MixedModelCall(guide,captured_gbc_table,sample,guide_calls))

256

In [6]:
##output table
pop['good_coverage']=True
pop['number_of_cells']=1.0
pop[['cell_barcode','guide_identity','read_count','UMI_count','coverage','gemgroup','good_coverage','number_of_cells']].to_csv(output_dir+'cell_identities.csv',index=False)
